In [4]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.endpoints.instruments import InstrumentsCandles
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import talib
import pandas_ta as ta
import numpy as np
import pandas as pd
import random


In [5]:
def signal_generator(df):
    vwap = df['VWAP']
    rsi = df['RSI'].dropna()

    close_price = df['Close']
    upper_bb = df['UpperBB']
    lower_bb = df['LowerBB']
    average_rsi = rsi.mean()
    bullish = True
    bearish = True
    sell_count = 0
    buy_count = 0
    #print(rsi.values)
    vwap_threshold = 0.0004
    for i in range(len(df)):
        if max(df['Open'][i], df['Close'][i]) >= (1 + vwap_threshold) * df['VWAP'][i]:
            bearish = False
        if min(df['Open'][i], df['Close'][i]) <= (1 - vwap_threshold) * df['VWAP'][i]:
            bullish = False

    if bullish and bearish:
        vwap_signal =  "Neutral"  # Neither Bullish nor Bearish
    elif bullish:
        vwap_signal =  "Bullish"
        buy_count += 1
    elif bearish:
        vwap_signal =  "Bearish"
        sell_count += 1
    else:
        vwap_signal =  "Neutral"

    if average_rsi > 70:
        rsi_signal = "Overbought"
        sell_count += 1
    elif average_rsi < 30:
        rsi_signal = "Oversold"
        buy_count += 1
    else:
        rsi_signal = "Neutral"
        
    bb_threshold = 0.2
    overbought_signal = (df['Close'] > upper_bb).any()
    oversold_signal = (df['Close'] < lower_bb).any()

    if overbought_signal:
        bb_signal = "Overbought"
        sell_count += 1
    elif oversold_signal:
        bb_signal = "Oversold"
        buy_count += 1
    else:
        bb_signal = "Neutral"

    if buy_count >= 2:
        #print(vwap_signal, rsi_signal, bb_signal )
        return "Buy"
    elif sell_count >= 2:
        #print(vwap_signal, rsi_signal, bb_signal )
        return "Sell"
    else:
        return "-"
    


### 3 - Connect to the market and execute trades

In [6]:
access_token= "cd7e21b015cb734770f5fcfea9da9c9f-49a95de76ed5cab29ee0d33cee5f004d"
accountID = "101-001-27332384-001" #your account ID here
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
def get_candles(n, currency):
    client = API(access_token)
    request = InstrumentsCandles(instrument=currency, params={"granularity": Gran.M15, "count": n})
    response = client.request(request)
    candles = response['candles']
    return candles

candles = get_candles(3, Pair.EUR_USD)
candles


[{'complete': True,
  'volume': 627,
  'time': '2023-12-04T03:30:00.000000000Z',
  'mid': {'o': '1.08856', 'h': '1.08860', 'l': '1.08836', 'c': '1.08848'}},
 {'complete': True,
  'volume': 486,
  'time': '2023-12-04T03:45:00.000000000Z',
  'mid': {'o': '1.08849', 'h': '1.08854', 'l': '1.08801', 'c': '1.08801'}},
 {'complete': False,
  'volume': 349,
  'time': '2023-12-04T04:00:00.000000000Z',
  'mid': {'o': '1.08800', 'h': '1.08806', 'l': '1.08765', 'c': '1.08771'}}]

In [7]:
def calculate_vwap(data):
    data['VWAP'] = (((data['High'] + data['Low'] + data['Close']) / 3) * data['Volume']).cumsum() / data['Volume'].cumsum()

def calculate_bollinger_bands(data, std_dev=2):
    data['SMA'] = data['Close'].mean()
    std_dev_values = data['Close'].std()
    data['UpperBB'] = data['SMA'] + (std_dev * std_dev_values)
    data['LowerBB'] = data['SMA'] - (std_dev * std_dev_values)
    
def calculate_rsi(data):
    data['RSI'] = talib.RSI(data['Close'])

In [8]:
def trading_job(currency, string_currency):
    candles = get_candles(15, currency)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low', 'Volume'])
    i=0

    for candle in candles:
        dfstream.loc[i, 'Open'] = float(candle['mid']['o'])
        dfstream.loc[i, 'Close'] = float(candle['mid']['c'])
        dfstream.loc[i, 'High'] = float(candle['mid']['h'])
        dfstream.loc[i, 'Low'] = float(candle['mid']['l'])
        dfstream.loc[i, 'Volume'] = float(candle['volume'])
        i += 1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)
    dfstream['Volume'] = dfstream['Volume'].astype(float)
    calculate_vwap(dfstream)
    calculate_bollinger_bands(dfstream)
    calculate_rsi(dfstream)

    #print(dfstream)
    #print(dfstream)
    signal = signal_generator(dfstream)#
    # EXECUTING ORDERS
    #print("signal: " + str(signal))
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle['mid']['o']))-previous_candleR
    SLSell = float(str(candle['mid']['o']))+previous_candleR

    TPBuy = float(str(candle['mid']['o']))+previous_candleR*SLTPRatio
    TPSell = float(str(candle['mid']['o']))-previous_candleR*SLTPRatio

    #print(dfstream.iloc[:-1,:])
    #print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2

    #THE WRONG CODE SELL SOULD BE -2000 and BUY SHOULD BE 2000
    #Sell
    if signal == "Sell":
        random_integer = random.randint(-100, 100)
        mo = MarketOrderRequest(instrument=string_currency, units=2000 + random_integer, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == "Buy":
        random_integer = random.randint(-100, 100)
        mo = MarketOrderRequest(instrument=string_currency, units=-2000 + random_integer, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [9]:

currency_pairs = [Pair.AUD_CAD, Pair.AUD_JPY, Pair.AUD_NZD, Pair.AUD_USD,
    Pair.CAD_CHF, Pair.CAD_JPY,
    Pair.EUR_AUD, Pair.EUR_CAD, Pair.EUR_CHF, Pair.EUR_GBP, Pair.EUR_JPY, Pair.EUR_NZD, Pair.EUR_USD,
    Pair.GBP_AUD, Pair.GBP_CAD, Pair.GBP_CHF, Pair.GBP_JPY, Pair.GBP_NZD, Pair.GBP_USD,
    Pair.NZD_CAD, Pair.NZD_CHF, Pair.NZD_JPY, Pair.NZD_USD,
    Pair.USD_CAD, Pair.USD_CHF, Pair.USD_JPY]

list_of_str_currency = ["AUD_CAD", "AUD_JPY", "AUD_NZD", "AUD_USD",
    "CAD_CHF", "CAD_JPY",
    "EUR_AUD", "EUR_CAD", "EUR_CHF", "EUR_GBP", "EUR_JPY", "EUR_NZD", "EUR_USD",
    "GBP_AUD", "GBP_CAD", "GBP_CHF", "GBP_JPY", "GBP_NZD", "GBP_USD",
    "NZD_CAD", "NZD_CHF", "NZD_JPY", "NZD_USD",
    "USD_CAD", "USD_CHF", "USD_JPY"]


In [10]:
# for i in range (len(list_of_str_currency)):
#     trading_job(currency_pairs[i],list_of_str_currency
# [i])

scheduler = BlockingScheduler()
for i in range (len(currency_pairs)):
    scheduler.add_job(trading_job, 'cron', args = [currency_pairs[i],list_of_str_currency[i]], day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00')
scheduler.start()


Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 00:01:00 PST)" was missed by 0:00:40.370600
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 09:01:00 PST)" was missed by 0:00:40.380668
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 09:01:00 PST)" was missed by 0:00:40.458439
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 09:01:00 PST)" was missed by 0:00:40.461832
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 09:01:00 PST)" was missed by 0:00:40.462822
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 09:01:00 PST)" was misse

{'orderCreateTransaction': {'id': '2092', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2092', 'requestID': '79195083779274305', 'time': '2023-12-04T17:16:02.198494116Z', 'type': 'MARKET_ORDER', 'instrument': 'NZD_CHF', 'units': '2089', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '0.53804', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '0.53879', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '2093', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2092', 'requestID': '79195083779274305', 'time': '2023-12-04T17:16:02.198494116Z', 'type': 'ORDER_CANCEL', 'orderID': '2092', 'reason': 'TAKE_PROFIT_ON_FILL_LOSS'}, 'relatedTransactionIDs': ['2092', '2093'], 'lastTransactionID': '2093'}
{'orderCreateTransaction': {'id': '2094', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2094', 'requestID': '43166290528127493', 'time': '

Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 12:16:00 PST)" was missed by 0:00:38.380142
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 12:16:00 PST)" was missed by 0:00:38.383491
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 12:16:00 PST)" was missed by 0:00:38.389288
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 12:16:00 PST)" was missed by 0:00:38.386661
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 12:16:00 PST)" was missed by 0:00:38.391006
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 12:16:00 PST)" was misse

{'orderCreateTransaction': {'id': '2158', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2158', 'requestID': '79195155498053507', 'time': '2023-12-04T22:01:01.660674794Z', 'type': 'MARKET_ORDER', 'instrument': 'AUD_JPY', 'units': '-1981', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '97.498', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '97.423', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '2159', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2158', 'requestID': '79195155498053507', 'time': '2023-12-04T22:01:01.660674794Z', 'type': 'ORDER_CANCEL', 'orderID': '2158', 'reason': 'MARKET_HALTED'}, 'relatedTransactionIDs': ['2158', '2159'], 'lastTransactionID': '2159'}
{'orderCreateTransaction': {'id': '2160', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2160', 'requestID': '43166362251343807', 'time': '2023-12-04T2

Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 15:01:00 PST)" was missed by 0:10:20.995759
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 15:01:00 PST)" was missed by 0:10:21.001637
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 15:01:00 PST)" was missed by 0:10:21.002293
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 15:01:00 PST)" was missed by 0:10:21.003250
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 15:01:00 PST)" was missed by 0:10:21.003388
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 15:01:00 PST)" was misse

...

{'orderCreateTransaction': {'id': '2218', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2218', 'requestID': '43166392452470970', 'time': '2023-12-05T00:16:01.977047228Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_CHF', 'units': '-2065', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '0.94634', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '0.94469', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '2219', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2218', 'requestID': '43166392452470970', 'time': '2023-12-05T00:16:01.977047228Z', 'type': 'ORDER_CANCEL', 'orderID': '2218', 'reason': 'TAKE_PROFIT_ON_FILL_LOSS'}, 'relatedTransactionIDs': ['2218', '2219'], 'lastTransactionID': '2219'}


Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 16:46:00 PST)" was missed by 0:00:01.003785
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 16:46:00 PST)" was missed by 0:00:01.016573
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 16:46:00 PST)" was missed by 0:00:01.024675
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 16:46:00 PST)" was missed by 0:00:01.034705
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 16:46:00 PST)" was missed by 0:00:01.035716
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 16:46:00 PST)" was misse

{'orderCreateTransaction': {'id': '2220', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2220', 'requestID': '43166396228222485', 'time': '2023-12-05T00:31:01.050717313Z', 'type': 'MARKET_ORDER', 'instrument': 'AUD_CAD', 'units': '1956', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '0.89475', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '0.89628', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '2221', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2220', 'requestID': '43166396228222485', 'time': '2023-12-05T00:31:01.050717313Z', 'type': 'ORDER_CANCEL', 'orderID': '2220', 'reason': 'TAKE_PROFIT_ON_FILL_LOSS'}, 'relatedTransactionIDs': ['2220', '2221'], 'lastTransactionID': '2221'}


Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 17:01:00 PST)" was missed by 0:00:01.066249
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 17:01:00 PST)" was missed by 0:00:01.087527
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 17:01:00 PST)" was missed by 0:00:01.094213
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 17:01:00 PST)" was missed by 0:00:01.103686
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 17:01:00 PST)" was missed by 0:00:01.127651
Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2023-12-04 17:01:00 PST)" was misse

{'orderCreateTransaction': {'id': '2222', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2222', 'requestID': '43166400003996561', 'time': '2023-12-05T00:46:01.496541962Z', 'type': 'MARKET_ORDER', 'instrument': 'GBP_USD', 'units': '1975', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.26287', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.26491', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '2223', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2222', 'requestID': '43166400003996561', 'time': '2023-12-05T00:46:01.496541962Z', 'type': 'ORDER_CANCEL', 'orderID': '2222', 'reason': 'TAKE_PROFIT_ON_FILL_LOSS'}, 'relatedTransactionIDs': ['2222', '2223'], 'lastTransactionID': '2223'}
{'orderCreateTransaction': {'id': '2224', 'accountID': '101-001-27332384-001', 'userID': 27332384, 'batchID': '2224', 'requestID': '25152005261519839', 'time': '

In [ ]:
da= pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
da = da[:10000]
signals = []

for i in range(0, len(da)):
    
    block = da.iloc[i:i+15].copy()
    calculate_vwap(block)
    calculate_bollinger_bands(block)
    calculate_rsi(block)
    block = block.reset_index(drop=True)
    if not block.empty:
        signal = signal_generator(block)
        signals.append(signal)
x = pd.Series(signals)
x.value_counts()



-       9209
Buy      397
Sell     394
Name: count, dtype: int64

In [8]:
#consider making 65 / 35 RSI + 1 and make above 70 / 30 as + 2 ssss